This notebook considers: Interpolation
1. __data.x1__ in data.py. It contains 23 features (initiator exclusive). 
2. From the result, we can find there is no difference with manual selected 23 features. We don't generate features in this notebook but we get rules for 23 features.
3. In this notebook, we compare the prediction order (sphere, worm, vesicle, other) with (vesicle, worm, sphere, other). There is no difference between select variables.

In [1]:
import data1 as data
from common import *
from rules import *
from realkd.patch import RuleFit
from sklearn.model_selection import cross_validate, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneOut
import numpy as np
import matplotlib.colors as mcolorss
import statsmodels.api as sm
from linear_models import *
import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.multioutput import ClassifierChain
from multilabel import BinaryRelevanceClassifier, ProbabilisticClassifierChain

STATE = np.random.RandomState(seed=1)

lr = LogisticRegressionCV(penalty='l1', solver='saga', random_state=STATE)
lr_ind = BinaryRelevanceClassifier(lr)
lr_chain = ClassifierChain(lr, order=[0, 2, 1, 3])
lr_pcc = ProbabilisticClassifierChain(lr) 


glm_pcc = ProbabilisticClassifierChain(GlmWrapper(intercept=True),glm_intercept=True) 

rf = RandomForestClassifier(random_state=STATE, min_samples_leaf=1, n_estimators=100)
rf_ind = BinaryRelevanceClassifier(rf)
rf_chain = ClassifierChain(rf, order=[0, 2, 1, 3])
rf_pcc = ProbabilisticClassifierChain(rf)

# Rulefit
rufit_pcc = RuleFitWrapper(mode='chain')

full_estimators = [glm_pcc] #[lr_ind, lr_pcc, lr_chain, glm_pcc]#, rf_ind, rf_pcc, rf_chain, rufit_pcc]
full_names = ['Glm_pcc'] #['LR_ind', 'LR_pcc', 'LR_chain', 'Glm_pcc']#, 'RanF_ind', 'RanF_pcc', 'Ranf_chain', 'Rufit_pcc']

In [ ]:
from common import Experiment, LogLikelihoodEvaluator
from sklearn.model_selection import KFold

print("Current Prediction Order is:", data.y.columns.tolist())
print('Num of predictors:, ', data.x1.shape[1])

for STATE in range(22, 10000):
    try:
        interpolation = Experiment(full_estimators, 
                            full_names,
                            KFold(30, shuffle=True, random_state=STATE),
                            data.x1, data.y.replace(-1.0, 0.0),
                            groups=data.comp_ids.array, 
                            evaluators=['accuracy', LogLikelihoodEvaluator(base=2)],
                            verbose=True, file_name='interpolation_full').run()
        print(STATE)
        break
    except:
        pass

Current Prediction Order is: ['sphere', 'worm', 'vesicle', 'other']
Num of predictors:,  23
Running experiment with 30 repetitions
*******Running experiment with 30 repetitions
*******************Running experiment with 30 repetitions
******Running experiment with 30 repetitions
**********Running experiment with 30 repetitions
******************Running experiment with 30 repetitions
*********************Running experiment with 30 repetitions
*******Running experiment with 30 repetitions
**************Running experiment with 30 repetitions
****Running experiment with 30 repetitions
*********Running experiment with 30 repetitions
Running experiment with 30 repetitions
*Running experiment with 30 repetitions
***************************Running experiment with 30 repetitions
Running experiment with 30 repetitions
*********Running experiment with 30 repetitions
***Running experiment with 30 repetitions
************Running experiment with 30 repetitions
*******Running experiment with 30 repet

Running experiment with 30 repetitions
Running experiment with 30 repetitions
Running experiment with 30 repetitions
Running experiment with 30 repetitions
Running experiment with 30 repetitions
Running experiment with 30 repetitions
Running experiment with 30 repetitions
Running experiment with 30 repetitions
Running experiment with 30 repetitions
Running experiment with 30 repetitions
Running experiment with 30 repetitions
Running experiment with 30 repetitions
Running experiment with 30 repetitions
Running experiment with 30 repetitions
Running experiment with 30 repetitions
Running experiment with 30 repetitions


In [ ]:
STATE

In [ ]:
interpolation.summary()

In [4]:
# import pickle
# with open('interpolation_full_phase.pkl', 'wb') as f:   
#     pickle.dump(interpolation, f)

In [3]:
import pickle
with open('interpolation_full_phase.pkl', 'rb') as f:   
    interpolation  = pickle.load(f)

In [33]:
interpolation.fitted_['LR_pcc'][0].baselearner

LogisticRegressionCV(penalty='l1',
                     random_state=RandomState(MT19937) at 0x7F9A29AE5C40,
                     solver='saga')

In [4]:
interpolation.summary()

,mean_train_accuracy,std_train_accuracy,mean_test_accuracy,std_test_accuracy,mean_train_log likelihood,std_train_log likelihood,mean_test_log likelihood,std_test_log likelihood
LR_ind,0.382689,0.011744,0.378509,0.149020,-2.908712,0.011742,-2.929920,0.345263
LR_pcc,0.382689,0.011744,0.378509,0.149020,-2.908712,0.011742,-2.929920,0.345263
LR_chain,0.382689,0.011744,0.378509,0.149020,-2.908712,0.011742,-2.929920,0.345263
RanF_ind,0.999825,0.000533,0.783421,0.070991,-0.260965,0.004806,-1.201244,0.810822
RanF_pcc,0.999185,0.001356,0.800789,0.066668,-0.245952,0.004644,-1.163131,0.806772
Ranf_chain,1.000000,0.000000,0.805614,0.067503,-0.246824,0.004042,-1.119475,0.721598
Rufit_pcc,0.954916,0.005991,0.785439,0.090540,-0.374219,0.011522,-1.039242,0.431893


In [5]:
inter_no_comp_df = interpolation.summary()
change_columns = {"mean_train_log likelihood": 'mean_train_log loss', 
                  'std_train_log likelihood': 'std_train_log loss',
                  'mean_test_log likelihood': 'mean_test_log loss',
                  'std_test_log likelihood': 'std_test_log loss'} # chang log likelihood to log loss

inter_no_comp_df = inter_no_comp_df.rename(change_columns, axis=1)
inter_no_comp_df['mean_train_log loss'] = -1 * inter_no_comp_df['mean_train_log loss']
inter_no_comp_df['mean_test_log loss'] = -1 * inter_no_comp_df['mean_test_log loss']
   
inter_no_comp_df = inter_no_comp_df[inter_no_comp_df.index.isin(['LR_ind', 'RanF_pcc', 'Rufit_pcc'])]
inter_no_comp_df.index = ['LR', 'Rf', 'Rufit']
inter_no_comp_df

,mean_train_accuracy,std_train_accuracy,mean_test_accuracy,std_test_accuracy,mean_train_log loss,std_train_log loss,mean_test_log loss,std_test_log loss
LR,0.382689,0.011744,0.378509,0.149020,2.908712,0.011742,2.929920,0.345263
Rf,0.999185,0.001356,0.800789,0.066668,0.245952,0.004644,1.163131,0.806772
Rufit,0.954916,0.005991,0.785439,0.090540,0.374219,0.011522,1.039242,0.431893


In [6]:
df_gam = pd.read_csv('inter_GAM_result.csv')
df_gam = df_gam.rename(columns = {'mean_train_logloss': 'mean_train_log loss',
                        'std_train_logloss': 'std_train_log loss',
                        'mean_test_logloss': 'mean_test_log loss',
                        'std_test_logloss': 'std_test_log loss'})
df_gam.set_index('Unnamed: 0', inplace=True)
df_gam.index.name = None
df_gam.head()

,mean_train_accuracy,std_train_accuracy,mean_test_accuracy,std_test_accuracy,mean_train_log loss,std_train_log loss,mean_test_log loss,std_test_log loss
joint,0.672152,0.103714,0.605526,0.103714,3.431240,0.238151,4.864206,2.329445
sphere,0.862069,0.004505,0.844386,0.084767,0.505948,0.009251,0.590962,0.301794
vesicle,0.882548,0.003317,0.857368,0.066238,0.378220,0.006330,0.457261,0.230729
worm,0.816908,0.003732,0.787719,0.089400,0.579386,0.006241,0.642932,0.213112
other,0.986697,0.001697,0.971053,0.037293,0.051239,0.007446,0.761356,1.540761


In [7]:
temp = df_gam.loc[df_gam.index=='joint', ]
temp.index = ['Gam']      
inter_no_comp_df = pd.concat([inter_no_comp_df, temp])
inter_no_comp_df

,mean_train_accuracy,std_train_accuracy,mean_test_accuracy,std_test_accuracy,mean_train_log loss,std_train_log loss,mean_test_log loss,std_test_log loss
LR,0.382689,0.011744,0.378509,0.149020,2.908712,0.011742,2.929920,0.345263
Rf,0.999185,0.001356,0.800789,0.066668,0.245952,0.004644,1.163131,0.806772
Rufit,0.954916,0.005991,0.785439,0.090540,0.374219,0.011522,1.039242,0.431893
Gam,0.672152,0.103714,0.605526,0.103714,3.431240,0.238151,4.864206,2.329445


In [8]:
inter_no_comp_df['mean_train_error'] = 1-inter_no_comp_df['mean_train_accuracy']
inter_no_comp_df['std_train_error'] = inter_no_comp_df['std_train_accuracy']
inter_no_comp_df['mean_test_error'] = 1-inter_no_comp_df['mean_test_accuracy']
inter_no_comp_df['std_test_error'] = inter_no_comp_df['std_test_accuracy']
inter_no_comp_df = inter_no_comp_df.reindex(['LR', 'Gam', 'Rufit', 'Rf'])

inter_no_comp_df

,mean_train_accuracy,std_train_accuracy,mean_test_accuracy,std_test_accuracy,mean_train_log loss,std_train_log loss,mean_test_log loss,std_test_log loss,mean_train_error,std_train_error,mean_test_error,std_test_error
LR,0.382689,0.011744,0.378509,0.149020,2.908712,0.011742,2.929920,0.345263,0.617311,0.011744,0.621491,0.149020
Gam,0.672152,0.103714,0.605526,0.103714,3.431240,0.238151,4.864206,2.329445,0.327848,0.103714,0.394474,0.103714
Rufit,0.954916,0.005991,0.785439,0.090540,0.374219,0.011522,1.039242,0.431893,0.045084,0.005991,0.214561,0.090540
Rf,0.999185,0.001356,0.800789,0.066668,0.245952,0.004644,1.163131,0.806772,0.000815,0.001356,0.199211,0.066668


In [9]:
inter_no_comp_df.to_csv('inter_full_performance.csv')

In [ ]:
def plot_summary(metric, summ, num_reps=30, baseline=None, names=None, colors = list(mcolors.BASE_COLORS.keys())):
    width = 0.35
    ind = np.arange(len(summ))
    plt.bar(ind-width/2, summ[f'mean_train_{metric}'], width=width, label='train', 
            yerr=summ[f'std_train_{metric}']/num_reps**0.5, capsize=3.0)
    plt.bar(ind+width/2, summ[f'mean_test_{metric}'], width=width, label='test',
            yerr=summ[f'std_test_{metric}']/num_reps**0.5, capsize=3.0)
    if baseline:
        for i in range(len(baseline)):
            plt.axhline(y=baseline[i], color=colors[i], linestyle='-', label=names[i])
    plt.ylabel(metric)
    plt.legend()
    plt.xticks(ind, summ.index)